In [3]:
import pandas as pd
import numpy as np
import json
import pprint
import requests
from bs4 import BeautifulSoup
pd.set_option('display.width', 80)
pd.set_option('display.max_columns', 6)

In [4]:
webpage = requests.get("http://www.alrb.org/datacleaning/covidcaseoutliers.html")
bs = BeautifulSoup(webpage.text, 'html.parser')
theadrows = bs.find('table', {'id':'tblDeaths'}).thead.find_all('th')
type(theadrows)

bs4.element.ResultSet

In [5]:
labelcols = [j.get_text() for j in theadrows]
labelcols[0] ="rowheadings"
labelcols

['rowheadings',
 'Cases',
 'Deaths',
 'Cases per Million',
 'Deaths per Million',
 'population',
 'population_density',
 'median_age',
 'gdp_per_capita',
 'hospital_beds_per_100k']

In [6]:
rows = bs.find('table', {'id':'tblDeaths'}).tbody.find_all('tr')
datarows = []
labelrows = []
for row in rows:
    rowlabels = row.find('th').get_text()
    cells = row.find_all('td', {'class':'data'})
    if (len(rowlabels)>3):
        labelrows.append(rowlabels)
    if (len(cells)>0):
        cellvalues = [j.get_text() for j in cells]
        datarows.append(cellvalues)
pprint.pprint(datarows[0:2])

[['9,394', '653', '214', '15', '43,851,043', '17', '29', '13,914', '1.9'],
 ['16,642', '668', '1848', '74', '9,006,400', '107', '44', '45,437', '7.4']]


In [7]:
pprint.pprint(labelrows[0:2])

['Algeria', 'Austria']


In [8]:
for i in range(len(datarows)):
    datarows[i].insert(0, labelrows[i])
pprint.pprint(datarows[0:2])

[['Algeria',
  '9,394',
  '653',
  '214',
  '15',
  '43,851,043',
  '17',
  '29',
  '13,914',
  '1.9'],
 ['Austria',
  '16,642',
  '668',
  '1848',
  '74',
  '9,006,400',
  '107',
  '44',
  '45,437',
  '7.4']]


In [9]:
totaldeaths = pd.DataFrame(datarows, columns=labelcols)
totaldeaths.iloc[:, 1:5].head()
totaldeaths.dtypes

rowheadings               object
Cases                     object
Deaths                    object
Cases per Million         object
Deaths per Million        object
population                object
population_density        object
median_age                object
gdp_per_capita            object
hospital_beds_per_100k    object
dtype: object

In [10]:
totaldeaths.columns = totaldeaths.columns.str.replace(" ", "_").str.lower()

In [12]:
for col in totaldeaths.columns[1:-1]:
    totaldeaths[col] = totaldeaths[col].str.replace("[^0-9]","").astype('int64')

In [13]:
totaldeaths['hospital_beds_per_100k'] = totaldeaths['hospital_beds_per_100k'].astype('float')
totaldeaths.head()

,rowheadings,cases,deaths,...,median_age,gdp_per_capita,hospital_beds_per_100k
0,Algeria,9394,653,...,29,13914,1.9
1,Austria,16642,668,...,44,45437,7.4
2,Bangladesh,47153,650,...,28,3524,0.8
3,Belgium,58381,9467,...,42,42659,5.6
4,Brazil,514849,29314,...,34,14103,2.2


In [14]:
totaldeaths.dtypes

rowheadings                object
cases                       int64
deaths                      int64
cases_per_million           int64
deaths_per_million          int64
population                  int64
population_density          int64
median_age                  int64
gdp_per_capita              int64
hospital_beds_per_100k    float64
dtype: object